In [1]:
import os
import re
import pandas as pd
from collections import defaultdict
from extract import extract_relevant_content

In [2]:
gold_standard_df = pd.read_excel('./gold standard.xlsx', index_col=None)  
gold_standard_df = gold_standard_df.fillna("")
gold_standard_df

,Paper,Study design,Target population,Financial detail/costs,Person-level outcomes
0,#2598,This study used a sequential explanatory equal...,family- and center-based child care programs. ...,Five of the eight center directors not affilia...,
1,#17192,,,,"Of those 325 children, 59% (58.77%) were consi..."
2,#17247,bringing together the findings from interviews...,Indigenous Asháninka people of the Ene River i...,"OUTCOME FUNDS (USD) $110,000 // PAYMENT SCHEDU...",Results per outcome metric: 1) 75% of target a...
3,#17284,"We show, through three ECE case studies, how U...","3,500 students in Salt Lake City // 2,618 new ...","In the SIB contract, success payments are expe...","In the first cohort, 595 students attended pre..."
4,#17725,,,,Outcome 1: Learning gains\nBy the end of Year ...
5,#17755,The focus of the evaluation is the DIBs fundin...,"more than 300,000 primary school aged children...",Up to CHF 26.09 million of outcome payments wi...,


In [3]:
target_papers = ["#2598", "#17192", "#17247", "#17284", "#17725", "#17755"]

In [4]:
from collections import defaultdict
gold_standard = defaultdict(dict)
for i, row in gold_standard_df.iterrows():
    paper = row["Paper"]
    sd = [sample.strip() for sample in row["Study design"].split("//")]
    tp = [sample.strip() for sample in row["Target population"].split("//")]
    fd = [sample.strip() for sample in row["Financial detail/costs"].split("//")]
    po = [sample.strip() for sample in row["Person-level outcomes"].split("//")]
    gold_standard["Study Design"][paper] = sd
    gold_standard["Target Population"][paper] = tp
    gold_standard["Financial detail and costs"][paper] = fd
    gold_standard["Person-level outcomes"][paper] = po

In [5]:
path = "../data/samples"
articles = defaultdict(list)
for f in os.listdir(path):
    if f[0] == ".":
        continue
    id_ = f.split("-")[0].strip()
    print(os.path.join(path, f), id_)
    passages = extract_relevant_content(os.path.join(path,f))
    passages = [passage for passage in passages if len(passage.split()) <= 300 and len(passage.split()) > 10]
    # for finicial target
    # passages = [passage for passage in passages if len(passage.split()) <= 300 and len(passage.split()) > 10 and (re.search("[£$]\d+", passage) or re.search("\d,\d\d\d", passage) or re.search("\d+(k|mil|%)", passage))]
    articles[id_].extend(passages)

../data/samples/#17755 - Ecorys 2019.pdf #17755
657742 -1 657971
Avg paragraph length:  122.428
5250
../data/samples/#2598 - Lee 2020.pdf #2598
96504 -1 110368
Avg paragraph length:  274.6034482758621
348
../data/samples/#17284 - Warner 2018.pdf #17284
56317 -1 70715
Avg paragraph length:  307.010989010989
182
../data/samples/#17725 - IDinsight 2018.pdf #17725
28878 29441 59124
Avg paragraph length:  239.8728813559322
118
Avg paragraph length:  29.298992161254198
893
../data/samples/#17247 - Brookings 2017.pdf #17247
103219 -1 290474
Avg paragraph length:  164.2778675282714
619
../data/samples/#17192 - Education 2016 (1).pdf #17192
44998 51396 106565
Avg paragraph length:  84.10384615384615
520
Avg paragraph length:  107.04970178926442
503


In [15]:
# we assume two sentences overlap if they satisified the following contditions: 
# 1) 50% of sent1 overlaps with sent2 
# or 
# 2) 50% of sent2 overlaps with sent1
def isOverlap(input_text, gold_text):
    A = input_text.lower().strip().split()
    B = gold_text.lower().strip().split()
    n = len(A)
    m = len(B)
 
    # Auxiliary dp[][] array
    dp = [[0 for i in range(m + 1)] for i in range(n + 1)]
 
    # Updating the dp[][] table
    # in Bottom Up approach
    for i in range(n - 1, -1, -1):
        for j in range(m - 1, -1, -1):
 
            # If A[i] is equal to B[i]
            # then dp[j][i] = dp[j + 1][i + 1] + 1
            if A[i] == B[j]:
                dp[i][j] = dp[i + 1][j + 1] + 1
    maxm = 0
    # Find maximum of all the values
    # in dp[][] array to get the
    # maximum length
    for i in dp:
        for j in i:
            # Update the length
            maxm = max(maxm, j)
 

    if maxm/n >= 0.5:
        return True
    elif maxm/m >= 0.5:
        return True
    else:
        return False

isOverlap("this is a test", "this is a test right")

True

In [17]:
question_type = "Person-level outcomes"
for id_ in articles:
    gold = gold_standard[question_type][id_]
    if gold == [""]:
        continue
    count = 0
    for g in gold:
#         print(id_ + " Gold:\n", g)
        exist = False
        for passage in articles[id_]:
            if isOverlap(passage, g):
                count += 1
                exist = True
#                 print("Passage:\n", passage)
                break
#         if not exist:
#             print("Passage: \nNone",)
#         print("\n***********\n")
    print(id_ + " " + str(count/len(gold)))

#17284 1.0
#17725 1.0
#17247 0.0
#17192 1.0


In [20]:
gold_standard[question_type]["#17247"]

['In the first cohort, 595 students attended preschool, 110 were deemed at-risk, but only one actually used special education in kindergarten (United Way of Salt Lake, 2015).']

In [26]:
articles["#17247"][0]
re.sub("\s*\n\s*", " ", articles["#17247"][0]).lower().strip()
# gold_standard[question_type]["#17247"]

'impact bonds in developing countries: early learnings from the field 2 | impact bonds in developing countries impact bonds in developing countries: early learnings from the field september 2017 emily gustafsson-wright is a fellow at the center for universal education at brookings izzy boggild-jones is a research analyst at the center for universal education at brookings dean segell is a manager at convergence justice durland is a knowledge associate at convergence acknowledgements the authors would like to thank many people for their contributions to this study. first, alison bukhari, toby eccles, safeena husain, jane newman, peter vanderwal and maya ziswiler for their helpful comments, feedback and insight on earlier drafts of the report. in addition, we would like to thank all those who supported with data collection for the deal book, and provided real time updates on the factsheets for each deal. we would also like to acknowledge those who participated in the impact bonds workshop